# GroupBy

Based on the QuantEcon open source code for economic modeling
https://datascience.quantecon.org/applications/

**Outcomes**

- Understand the split-apply-combine strategy for aggregate
  computations on groups of data  
- Be able use basic aggregation methods on `df.groupby` to compute
  within group statistics  
- Understand how to group by multiple keys at once  


**Data**

- Details for all delayed US domestic flights in December 2016,
  obtained from the [Bureau of Transportation
  Statistics](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time)  

In [1]:
#pip install nbconvert

In [2]:
#pip install pyppeteer

In [3]:
# Uncomment following line to install on colab
! pip install qeds

In [4]:
import random
import numpy as np
import pandas as pd
import qeds
import matplotlib.pyplot as plt

%matplotlib inline
# activate plot theme
import qeds

## Outline

- [GroupBy](#GroupBy)  
  - [Split-Apply-Combine](#Split-Apply-Combine)  
  - [Case Study: Airline Delays](#Case-Study:-Airline-Delays)  
  - [Exercises](#Exercises)  

## Split-Apply-Combine

One powerful paradigm for analyzing data is the “Split-Apply-Combine”
strategy.

This strategy has three steps:

1. `Split`: split the data into groups based on values in one or more columns.  
1. `Apply`: apply a function or routine to each group separately.  
1. `Combine`: combine the output of the apply step into a DataFrame,
  using the group identifiers as the index.  


We will cover the main components in this lecture, but we encourage you
to also study the [official
documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
to learn more about what is possible.

To describe the concepts, we will need some data.

We will begin with a simple made-up dataset to discuss the concepts and
then work through extended example and exercises with real data.

In [5]:
C = np.arange(1, 7, dtype=float)
C[[3, 5]] = np.nan
df = pd.DataFrame({
    "A" : [1, 1, 1, 2, 2, 2],
    "B" : [1, 1, 2, 2, 1, 1],
    "C": C,
})
df

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


### Simple Example

To perform the *Split* step, we call the `groupby` method on our
DataFrame.

The first argument to `groupby` is a description of how we want to
construct groups.

In the most basic version, we will pass a string identifying the column
name.

In [6]:
gbA = df.groupby("A")

The `type` of variable we get back is a `DataFrameGroupBy`, which we
will sometimes refer to as GroupBy for short.

In [7]:
type(gbA)

pandas.core.groupby.generic.DataFrameGroupBy

Looking at the “groups” inside of the GroupBy object can help us
understand what the GroupBy represents.

We can do this with the `gb.get_group(group_name)` method.

In [8]:
gbA.get_group(1)

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0


In [9]:
gbA.get_group(2)

,A,B,C
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


We can *apply* some of our favorite aggregation functions directly on the
`GroupBy` object.


If we pass a list of strings to `groupby`, it will group based on
unique combinations of values from all columns in the list.

Let’s see an example.

In [10]:
gbAB = df.groupby(["A", "B"])
type(gbAB)

pandas.core.groupby.generic.DataFrameGroupBy

In [11]:
gbAB.get_group((1, 1))

,A,B,C
0,1,1,1.0
1,1,1,2.0


Notice that we still have a GroupBy object, so we can apply our favorite
aggregations.

In [12]:
gbAB.count()

C
A B   
1 1  2
  2  1
2 1  1
  2  0

Notice that the output is a DataFrame with two levels on the index
and a single column `C`.

This highlights a principle of how pandas handles the *Combine* part of
the strategy:

> The index of the combined DataFrame will be the group identifiers,
with one index level per group key.

### Custom Aggregate Functions

So far, we have been applying built-in aggregations to our GroupBy object.

We can also apply custom aggregations to each group of a GroupBy in two
steps:

1. Write our custom aggregation as a Python function.  
1. Passing our function as an argument to the `.agg` method of a GroupBy.  


Let’s see an example.

In [13]:
def num_missing(df):
    "Return the number of missing items in each column of df"
    return df.isnull().sum()

We can call this function on our original DataFrame to get the number of
missing items in each column.

In [14]:
num_missing(df)

A    0
B    0
C    2
dtype: int64

We can also apply it to a GroupBy object to get the number of missing
items in each column *for each group*.

In [15]:
gbA.agg(num_missing)

,B,C
A,,
1,0,0
2,0,2


The key to keep in mind is that the function we pass to `agg` should
take in a DataFrame (or Series) and return a Series (or single value)
with one item per column in the original DataFrame.

When the function is called, the data for each group will be passed to
our function as a DataFrame (or Series).

### Transforms: The `apply` Method

We can apply transforms to DataFrames.

We can do the same with GroupBy objects using the `.apply` method.

Let’s see an example.

In [16]:
df

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


In [17]:
def smallest_by_b(df):
    return df.nsmallest(2, "B") #returns the first 2 rows by ascending order 
                                #of column "B" DataFrame.nsmallest(n, columns, keep='first')

In [18]:
gbA.apply(smallest_by_b)

A  B    C
A             
1 0  1  1  1.0
  1  1  1  2.0
2 4  2  1  5.0
  5  2  1  NaN

Notice that the return value from applying our series transform to `gbA`
was the group key on the outer level (the `A` column) and the original
index from `df` on the inner level.

The original index came along because that was the index of the
DataFrame returned by `smallest_by_b`.

Had our function returned something other than the index from `df`,
that would appear in the result of the call to `.apply`.


### `pd.Grouper`

Sometimes, in order to construct the groups you want, you need to give
pandas more information than just a column name.

Some examples are:

- Grouping by a column and a level of the index.  
- Grouping time series data at a particular frequency.  


pandas lets you do this through the `pd.Grouper` type.

To see it in action, let’s make a copy of `df` with `A` moved to the
index and a `Date` column added.

In [19]:
df2 = df.copy()
from datetime import datetime
df2["Date"] = pd.date_range(
    start=datetime.today().strftime("%m/%d/%Y"),
    freq="BQ", #business quarter end frequency
    periods=df.shape[0]
)
df2 = df2.set_index("A")
df2

,B,C,Date
A,,,
1,1,1.0,2023-03-31
1,1,2.0,2023-06-30
1,2,3.0,2023-09-29
2,2,NaN,2023-12-29
2,1,5.0,2024-03-29
2,1,NaN,2024-06-28


We can group by year.

In [20]:
df2.groupby(pd.Grouper(key="Date", freq="A")).count()

,B,C
Date,,
2023-12-31,4,3
2024-12-31,2,1


We can group by the `A` level of the index.

In [21]:
df2.groupby(pd.Grouper(level="A")).count()

,B,C,Date
A,,,
1,3,3,3
2,3,1,3


We can combine these to group by both.

In [22]:
df2.groupby([pd.Grouper(key="Date", freq="A"), pd.Grouper(level="A")]).count()

B  C
Date       A      
2023-12-31 1  3  3
           2  1  0
2024-12-31 2  2  1

And we can combine `pd.Grouper` with a string, where the string
denotes a column name

In [23]:
df2.groupby([pd.Grouper(key="Date", freq="A"), "B"]).count()

C
Date       B   
2023-12-31 1  2
           2  1
2024-12-31 1  1

## Exercises (GRADED: Submit on Crowdmark)



**Exercise 1**

Use introspection (tab completion) to see what other aggregations are defined for GroupBy objects.

Pick *three* and evaluate them in the cells below.

You can use the dataset below to attempt this exercise. 

In [24]:
# Creating a Dataset 
hogwarts = pd.DataFrame({
    "Character" : ["Harry", "Luna", "Cedric", "Hermione"],
    "House" : ["Gryffindor", "Ravenclaw", "Hufflepuff", "Gryffindor"]
})

SyntaxError: incomplete input (12706737.py, line 1)

In [ ]:
# method 2

In [ ]:
# method 3

**Exercise 2**

**Prelude: Which city produces the most bagels by year? Montreal or New York?**

This exercise has a few steps:

1. Write a function that, given a DataFrame, returns a DataFrame object with a new column that provides the maximum over the indices and columns chosen (see question 2 for an idea of how you will use your function)

2. Apply the function to the bagel dataset ( bagel ), where the indice chosen is time

3. Produce a new DataFrame with a variable called Champion that gives the city that produced the most bagel by year. The final DataFrame should have 4 columns: City, Year, BagelProduced and Champion

In [ ]:
# Dataframe 
bagel = pd.DataFrame({
    "City" : ["Montreal","Montreal","Montreal", "Montreal", "Montreal","Montreal","New York", "New York", "New York", "New York", "New York", "New York"],
    "BagelProduced" : [5, 10, 20, 25, 30, 35, 3, 13, 23, 23, 33, 33],
    "Year": [2000, 2001, 2002, 2003, 2004, 2005, 2000, 2001, 2002, 2003, 2004, 2005]
})

# Look at dataframe
bagel

In [ ]:
#your code here
